In [77]:
import numpy as np
import cv2

In [78]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [79]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    #z=depth(i,j),x=(j-cx)*z/fx,y=(i-cy)*z/fy
    # for-loop is not allowed!!
    # ------------TODO --------------
    idx,idy=np.meshgrid(np.arange(depth.shape[0]),np.arange(depth.shape[1]),indexing='ij')
    z=depth[seg==255]
    x=(idy[seg==255]-K[0,2])*z/K[0,0]
    y=(idx[seg==255]-K[1,2])*z/K[1,1]
    pc=np.vstack((x,y,z)).T
    return pc

partial_pc = depth2pc(dpt, seg, K)
#print(partial_pc.shape)
# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [80]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------
one_way_CD = 0
distance=np.linalg.norm(partial_pc_sampled-full_pc_sampled,axis=1)
one_way_CD=np.array([np.mean(np.min(distance,axis=0))])
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD)

one way chamfer distance:  [0.00663034]
